In [2]:
import pandas as pd
import numpy as np

In [178]:
#alert_time train
alert_time = pd.read_csv('./training_first/train_x_alert_date.csv')
#alert_time test
test_alert_time = pd.read_csv('./training_first/public_x_alert_date.csv')
#y
y = pd.read_csv('./training_first/train_y_answer.csv')

#ccba
ccba = pd.read_csv('./training_first/public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv('./training_first/public_train_x_cdtx0001_full_hashed.csv')
#custinfo
custinfo = pd.read_csv('./training_first/public_train_x_custinfo_full_hashed.csv')
#dp
dp = pd.read_csv('./training_first/public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv('./training_first/public_train_x_remit1_full_hashed.csv')

#案件名單
doc = pd.read_csv('./training_first/預測的案件名單及提交檔案範例.csv')

In [179]:
#合併alert_key、date與label
#occupation_code missing補類別21
alert = pd.concat([alert_time,test_alert_time])
base = alert.merge(y, on = 'alert_key', how = 'left').merge(custinfo, on = 'alert_key', how = 'left')
base[['occupation_code']] = base[['occupation_code']].fillna(value=21.0)

In [180]:
#gen yymm
yymm = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
bins = [-np.inf, 29, 60, 90, 121, 152, 182, 213, 243, 274, 305, 333, 364, 393]
base['yymm'] = pd.cut(base['date'], bins, labels = yymm)
base

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,yymm
0,171189,0,0.0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4,1
1,171202,0,0.0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2,1
2,171599,0,0.0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4,1
3,171737,0,0.0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4,1
4,171142,0,0.0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,1
...,...,...,...,...,...,...,...,...,...
25746,364472,393,NaN,bdda3ef79c5f052fd281c9f28f50a25977444eceaf9751...,3,19.0,2390889.0,4,13
25747,364788,393,NaN,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,13
25748,364673,393,NaN,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,13
25749,364626,393,NaN,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,13


In [184]:
#training data只取每個月最後一筆，testing data取全部
base1 = base[~base.sar_flag.isna()].sort_values(['cust_id','date']).drop_duplicates(subset=['cust_id','yymm'], keep='last')
base2 = base[base.sar_flag.isna()]
base3 = pd.concat([base1,base2])
#排掉第1個月資料(dp資料不完整)
base3 = base3[base3.yymm>1]

In [186]:
#串dp近30天交易資料
base3['start_date'] = base3.date - 30
base3['L1W_date'] = base3.date - 7
base3['L2W_date'] = base3.date - 14
base3['L3W_date'] = base3.date - 21
base3['L4W_date'] = base3.date - 28
dp_base = base3.merge(dp, on = 'cust_id', how = 'left')
dp_base = dp_base[(dp_base.tx_date > dp_base.start_date) & (dp_base.tx_date <= dp_base.date)]

In [187]:
base3

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,yymm,start_date,L1W_date,L2W_date,L3W_date,L4W_date
4353,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,2,23,46,39,32,25
13458,271556,221,0.0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,3,4.0,629787.0,3,8,191,214,207,200,193
9716,240088,153,0.0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,3,15.0,2126903.0,4,6,123,146,139,132,125
16322,294760,263,0.0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,3,15.0,980597.0,4,9,233,256,249,242,235
5740,206471,78,0.0,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,2,13.0,406.0,5,3,48,71,64,57,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364472,393,NaN,bdda3ef79c5f052fd281c9f28f50a25977444eceaf9751...,3,19.0,2390889.0,4,13,363,386,379,372,365
25747,364788,393,NaN,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,13,363,386,379,372,365
25748,364673,393,NaN,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,13,363,386,379,372,365
25749,364626,393,NaN,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,13,363,386,379,372,365


In [188]:
dp_base

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,AGE,yymm,start_date,L1W_date,L2W_date,L3W_date,L4W_date,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM
0,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,2,23,46,39,32,25,CR,35.0,15.0,2.0,4720.0,27.995,20.0,NaN,NaN,0.0,1.0
17,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,2,23,46,39,32,25,DB,53.0,18.0,2.0,142.0,21.550,20.0,NaN,NaN,0.0,1.0
19,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,2,23,46,39,32,25,CR,53.0,16.0,2.0,1161.0,1.000,6.0,NaN,NaN,0.0,0.0
22,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,2,23,46,39,32,25,DB,53.0,18.0,2.0,10407.0,21.550,20.0,NaN,NaN,0.0,1.0
23,195913,53,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,2,23,46,39,32,25,DB,46.0,11.0,2.0,151.0,1.000,16.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533597,364986,393,NaN,bd47c67a9d9cdd26007df624ff2f0860526e1e09d3a409...,1,19.0,33565.0,3,13,363,386,379,372,365,DB,389.0,21.0,1.0,NaN,1.000,13.0,21.0,134.0,0.0,1.0
6533598,364986,393,NaN,bd47c67a9d9cdd26007df624ff2f0860526e1e09d3a409...,1,19.0,33565.0,3,13,363,386,379,372,365,DB,378.0,0.0,1.0,NaN,1.000,13.0,21.0,10.0,0.0,1.0
6533600,364986,393,NaN,bd47c67a9d9cdd26007df624ff2f0860526e1e09d3a409...,1,19.0,33565.0,3,13,363,386,379,372,365,DB,379.0,4.0,1.0,NaN,1.000,13.0,21.0,10.0,0.0,1.0
6533601,364986,393,NaN,bd47c67a9d9cdd26007df624ff2f0860526e1e09d3a409...,1,19.0,33565.0,3,13,363,386,379,372,365,DB,375.0,1.0,1.0,NaN,1.000,13.0,21.0,130.0,0.0,1.0


In [43]:
base.groupby('occupation_code').agg({'occupation_code':'count'})

,occupation_code
occupation_code,
0.0,2
1.0,220
2.0,130
3.0,314
4.0,555
5.0,946
6.0,1
7.0,103
8.0,135
